In [1]:
# Sentinel Hub Config

from env_vars import sentinel_hub_instance_id
from sentinelhub import SHConfig

# Import Area of Interest List

import pandas as pd
import json
from scripts.mgrs import encode,LLtoUTM


# Sentinel Hub Tile Look Up / Download

from sentinelhub import WebFeatureService, BBox, CRS, DataSource, AwsTileRequest


# Cloud Masking

import rasterio as rio
import numpy as np
import earthpy.mask as em

# Generate Product Detail DataFrame

import os
from glob import glob
import xml.etree.ElementTree as ET


# Sort / Organize Tiles by Individual Folders

from shutil import copyfile,rmtree

# Reproject Masked Files 

import gdal
from glob import glob

# Create Master Raster


# Extract Polygon crops from products

import pandas as pd
from shapely.geometry import Polygon
import geopandas as gpd
from geopandas import GeoDataFrame
import earthpy.spatial as es

# TIF to JPG

from PIL import Image


In [27]:
gdal.UseExceptions()

In [48]:
def add_trailing_slash(path):
    if path[-1] != '/':
        path += '/'
    return path

In [4]:
def create_dir(output_dir):
    # If the output folder doesn't exist, create it
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

In [2]:
def shub_connect(sentinel_hub_instance_id):

    INSTANCE_ID = sentinel_hub_instance_id  

    if INSTANCE_ID:
        config = SHConfig()
        config.instance_id = INSTANCE_ID
    else:
        config = None
        
    return config

In [3]:
config = shub_connect(sentinel_hub_instance_id)

In [11]:
def import_aois(csv_loc):    

    df_labels = pd.read_csv(csv_loc)
    df_labels = df_labels[["center-lat","center-long","polygon"]]

    polygons = []
    for polygon in df_labels["polygon"]:
        polygons.append(json.loads(polygon)["coordinates"])

    coordinates = []
    for items in polygons:
        for item in items:
            for lon_lat in item:
                coordinates.append(lon_lat)

    #bounding box

    min_lon = min([i[0] for i in coordinates])
    min_lat = min([i[1] for i in coordinates])
    max_lon = max([i[0] for i in coordinates])
    max_lat = max([i[1] for i in coordinates])

    bounding_box = min_lon,min_lat,max_lon,max_lat


    tiles = []
    for ll in coordinates:
        if encode(LLtoUTM(ll[1],ll[0]),1)[:-2] == "34MAC":
            print(ll[1],ll[0])

        tiles.append(encode(LLtoUTM(ll[1],ll[0]),1)[:-2])

    tiles = list(set(tiles))
    return bounding_box,tiles

In [15]:
bounding_box,tiles = import_aois("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv")

-2.084882 18.280563
-2.442688 18.188553
-2.552447 18.203316
-2.690318 18.17173
-2.708837 18.236275
-2.084882 18.280563
-2.707722 18.066244
-2.700863 18.086715
-2.705622 18.090663


In [16]:
bounding_box

(8.42823, -3.373256, 25.688438, 5.845887)

In [17]:
polygon = {"type":"Polygon","coordinates":[[[5.493164,8.276727],[5.449219,-5.703448],[31.376953,-4.959615],[31.157227,8.711359],[5.493164,8.276727]]]}

items = json.loads(str(polygon["coordinates"]))
coordinates = []
for item in items:
            for lon_lat in item:
                coordinates.append(lon_lat)
#                 print(coordinates)
# tiles = []
# for ll in coordinates:
#     tiles.append(encode(LLtoUTM(ll[1],ll[0]),1)[:-2])
# #     print(ll[1],ll[0])


#bounding box

min_lon = min([i[0] for i in coordinates])
min_lat = min([i[1] for i in coordinates])
max_lon = max([i[0] for i in coordinates])
max_lat = max([i[1] for i in coordinates])

lon_list = []
bounding_box = min_lon,min_lat,max_lon,max_lat

for lon in np.arange(min_lon,max_lon,.05):
    lon_list.append(lon)
    
lat_list = []
for lat in np.arange(min_lat,max_lat,.05):
    lat_list.append(lat)

print(len(lon_list),len(lat_list))


lon_lat = []
for lon_2 in lon_list:
    for lat_2 in lat_list:
        lon_lat.append([lon_2,lat_2])

tiles_2 = []
for ll_2 in lon_lat:
    tiles.append(encode(LLtoUTM(ll_2[1],ll_2[0]),1)[:-2])



tile_list = list(set(tiles))
print(bounding_box,tile_list)

519 289
(5.449219, -5.703448, 31.376953, 8.711359) ['35MPP', '32MLC', '35MMT', '31MGP', '34MED', '33MYP', '35PJJ', '33NWJ', '32PNP', '35NLC', '34PGQ', '35NMJ', '32NKP', '33MWV', '32NRN', '34PDP', '33NVG', '35MJR', '35MRR', '35NNH', '34NAN', '31PGJ', '31NHG', '32NLN', '35NJF', '35NRB', '36NSL', '34MHD', '34MAA', '32NMP', '35MNU', '33NVF', '34MCV', '34MDE', '35NNJ', '35NPB', '32MLE', '32NJM', '34NAP', '36NTL', '35NMD', '33PTK', '33MZV', '35NJJ', '34MHA', '35NPD', '33NVB', '34MCA', '34NBH', '33NSD', '33MUR', '36NUN', '33NWE', '32PQP', '32NQM', '35MQT', '34NFP', '34NDG', '35NKB', '34NGP', '34MEE', '34PGP', '33NZA', '34NFN', '32NLJ', '32MRB', '35MKV', '36MUC', '32MNU', '32MNC', '34MHB', '33MZU', '34MCD', '31MHS', '34PCQ', '33MXQ', '33NVJ', '34NEN', '33NSE', '32NLK', '34NHM', '34NDN', '35NLG', '36NTP', '32PNQ', '32MMV', '32MRC', '33MYV', '33NVC', '32PMQ', '33MXU', '35MJP', '35PPJ', '33NXE', '32NJL', '31NHF', '35NNF', '31NHC', '34NBK', '33MST', '34MDC', '32MPA', '34PCP', '31MHV', '34NGH', '35

In [21]:
within_basin_tile_list = ['33MVV',
 '33NTE',
 '33NWE',
 '32NRK',
 '33NUB',
 '32NPG',
 '33NUC',
 '33NUG',
 '32NMK',
 '33NZD',
 '33NVE',
 '33NXD',
 '33NVD',
 '33NTG',
 '34NDF',
 '34MCE',
 '35NLB',
 '33NXE',
 '34MBC',
 '34MAC',
 '33NTC',
 '33NVF',
 '34MCB',
 '34NCH',
 '32NMM',
 '34NEH',
 '33MWV',
 '34NDG',
 '33NTB',
 '34NEF',
 '33NUD',
 '34NAJ',
 '33NWC',
 '34MAB',
 '34MBB',
 '32NQH',
 '33NYC',
 '33NWA',
 '33NUF',
 '34NDH',
 '33NWD',
 '33NUE',
 '33NWB',
 '33NVC',
 '33NYB',
 '33NTD',
 '34NFH',
 '32NNM',
 '32NRH',
 '33NXB',
 '33NYD',
 '33NVA',
 '33NSE',
 '33NSD',
 '34NFG',
 '34NGG',
 '33MZT',
 '32NQG',
 '33NSF',
 '33NXC',
 '33NYE',
 '33NTF',
 '32NML']

In [ ]:
# Congo Basin Bounding Box
search_bbox = BBox(bbox=[6.81,-5.27, 30, 5.5], crs=CRS.WGS84)

#Additional Params
bands = ['R10m/TCI']

In [11]:
def shub_lookup_tiles(bounding_box,tile_list,search_time_interval = ('2019-01-01T00:00:00', '2020-12-31T23:59:59'),
                   product_type = DataSource.SENTINEL2_L2A):
    
    #Misha's Tiles of Interest
    search_bbox = BBox(bbox=bounding_box, crs=CRS.WGS84)

    search_time_interval = ('2019-01-01T00:00:00', '2020-12-31T23:59:59')
    wfs_iterator = WebFeatureService(
        search_bbox,
        search_time_interval,
        data_source=product_type,
        maxcc=.05,
        config=config
    )
    results = wfs_iterator.get_tiles()
    df = pd.DataFrame(results, columns=['Tilename','Date','AmazonID'])
    df_tiles_of_interest = df[df["Tilename"].isin(tile_list)]
    df2 = df_tiles_of_interest.groupby('Tilename').head(10)
    output2 = list(df2.itertuples(index=False,name=None))
    return df,df2,output2

In [13]:
df,df2,output2 = shub_lookup_tiles()

NameError: name 'config' is not defined

In [123]:
df["Tilename"].value_counts()

35PNK    101
35PLK     96
35PPK     93
36MTV     93
35NNJ     88
        ... 
33NTC      1
32NPF      1
32MNE      1
32NPJ      1
33NTD      1
Name: Tilename, Length: 515, dtype: int64

In [125]:
df2["Tilename"].value_counts()

35NMJ    10
35NNC    10
33NZE    10
35NPD    10
34NEJ    10
         ..
32MNE     1
33NTC     1
32MQE     1
32NPF     1
32NPJ     1
Name: Tilename, Length: 515, dtype: int64

In [24]:
def shub_lookup_tiles(bounding_box,tile_list,search_time_interval = ('2019-01-01T00:00:00', '2020-12-31T23:59:59'),
                   product_type = DataSource.SENTINEL2_L2A):
    
    #Misha's Tiles of Interest
    search_bbox = BBox(bbox=bounding_box, crs=CRS.WGS84)

    search_time_interval = ('2019-01-01T00:00:00', '2020-12-31T23:59:59')
    wfs_iterator = WebFeatureService(
        search_bbox,
        search_time_interval,
        data_source=product_type,
        maxcc=.05,
        config=config
    )
    results = wfs_iterator.get_tiles()
    df = pd.DataFrame(results, columns=['Tilename','Date','AmazonID'])
    df_tiles_of_interest = df[df["Tilename"].isin(tile_list)]
    df2 = df_tiles_of_interest.groupby('Tilename').head(10)
    output2 = list(df2.itertuples(index=False,name=None))
    return output2

In [26]:
results_list = shub_lookup_tiles(bounding_box,tile_list,search_time_interval = ('2019-01-01T00:00:00', '2020-12-31T23:59:59'),
                   product_type = DataSource.SENTINEL2_L2A)

In [ ]:
def shub_download_tiles(results_list,output_dir,bands=["R10m/TCI"],product_type = DataSource.SENTINEL2_L2A):
    
    #Additional Params
    bands = bands
    
    output_dir = add_trailing_slash(output_dir)
    
    
    for tile in results_list:
        tile_name, time, aws_index = tile

        #Download SAFE Files
        request = AwsTileRequest(
            tile=tile_name,
            time=time,
            bands = bands, 
            aws_index=aws_index,
            data_folder=output_dir,
            data_source=product_type,
            safe_format = True
        )

        request.save_data(redownload=True)
    

In [ ]:
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test"

shub_download_tiles(results_list,output_dir,bands=["R10m/TCI"],product_type = DataSource.SENTINEL2_L2A)

In [188]:
def cloud_mask_tci(prod_dir):
    
    ''''
    
    prod refers product directory 
    
    ''''
    
    prod_dir = add_trailing_slash(prod_dir)
    
    msk_file_path = glob(src_dir + "*/*/MSK_CLDPRB_20m.jp2")[0]
    tci_file_path = glob(src_dir + "*/IMG_DATA/R10m/*.jp2")[0]
    tci_filename = tci_file_path.split("/")[-1]
    output_tci_file_path = src_dir + "/IMG_DATA/R10m/" + "processed_" + tci_filename 

    nodatavalue = int(0)

    with rio.open(tci_file_path) as sen_TCI_src:
        sen_TCI = sen_TCI_src.read(masked=True)
        sen_TCI_meta = sen_TCI_src.meta

    with rio.open(msk_file_path) as sen_mask_src:
        sen_mask_pre = sen_mask_src.read(1)
        sen_mask = np.repeat(np.repeat(sen_mask_pre,2,axis=0),2,axis=1)

    # All pixels above 0 probability will be classified as True

    sen_mask_qa = sen_mask > 0


    # Apply mask to source TCI file
    if np.count_nonzero(sen_mask_qa) > 0:
        sen_TCI_cl_free_nan = em.mask_pixels(sen_TCI, sen_mask_qa)
        sen_TCI_cl_free_processed = np.ma.filled(sen_TCI_cl_free_nan, fill_value=nodatavalue)
    else:
        sen_TCI_c1_free_processed = sen_mask_qa


    # Export cloud-masked TCI file
    with rio.open(output_tci_file_path, 'w', **sen_TCI_meta) as outf:
        outf.write(sen_TCI_cl_free_processed)

In [189]:
def apply_mask_tci_safe_list(products_dir):
    ''''
    
    products_dir refers to parent directory containing multiple products
    
    
    ''''
    
    products_dir = add_trailing_slash(products_dir)
    
    dir_list = glob(products_dir + "/*" )
    
    
    for directory in dir_list:
        cloud_mask_tci(directory)
        
    print(f"Applied masks to {len(dir_list)} products")

In [190]:
tci_folder_list = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_raw"
apply_mask_tci_safe_list(tci_folder_list)

Applied masks to 4 products


In [5]:
def generate_product_detail_df(input_dir):
    
    '''
    Generate product details dataframe used as input for ordering products by Cloudy Pixel Percentage, No Data Pixel Percentage, or Unclassified Percentage
    
    '''
    input_dir = add_trailing_slash(input_dir)
    
    dirs = os.listdir(input_dir)

    meta_data = []
    for folder in dirs:
        xml_loc = glob(input_dir + "/" + folder + "/*.xml")[0]
        tree = ET.parse(xml_loc)
        directory = [elem.text for elem in tree.iter() if "MASK_FILENAME" in elem.tag][0].split("/")[1]
        tile_id = directory.split("_")[1]
        filepath_partial = input_dir + "/" + directory + "/IMG_DATA" + "/R10m"
        filepath = glob(filepath_partial + "/processed*.jp2")[0]
        filename = filepath.split("/")[-1]
        cloud_cover,no_data,unclassified = [elem.text for elem in tree.iter() if "CLOUDY_PIXEL_PERCENTAGE" in elem.tag 
                 or "NODATA_PIXEL_PERCENTAGE" in elem.tag or "UNCLASSIFIED_PERCENTAGE" in elem.tag]
        meta_data.append([directory,tile_id,cloud_cover,no_data,unclassified,filename,filepath])
    df = pd.DataFrame(meta_data,columns=["Directory","Tile_Id","Cloud Cover","No Data Percentage","Unclassified Percentage","Filename","Filepath"])
    df2 = df.sort_values(by=["Tile_Id","Cloud Cover","Unclassified Percentage"],ignore_index=True)
    return df2

In [6]:
input_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_raw"

df = generate_product_detail_df(input_dir)


In [11]:
def order_masked_tiles(df,output_dir):
    
    '''
    
    df input is the products detail pre-sorted dataframe to be used for sorting products 
    
    '''
    
    output_dir = add_trailing_slash(output_dir)

    layer = 1
    for index,row in df.iterrows(): 
        destination_dir = output_dir + str(layer)
        output_file = destination_dir + "/" + row["Filename"]

        # Check if directory exists
        if not os.path.isdir(destination_dir):
            os.mkdir(destination_dir)

        # Copy file to existing or new directory
        copyfile(row["Filepath"],output_file)

        # Check if Tile_Id already exists in the directory - only necessary up until the last tile
        if len(df) > index + 1:
            if df.loc[index,"Tile_Id"] == df.loc[index + 1,"Tile_Id"]:
                layer += 1
            else:
                layer = 1 

In [12]:
order_masked_tiles(df,"/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_ordered/")

In [13]:
def csv_to_gdf(csv_loc):
    '''
    import manually created areas of interest csv
    
    output is an in-memory geo dataframe with one polygon AOI per row to be utilized for cropping master raster
    
    '''
    df = pd.read_csv(csv_loc)
    df_labels = df[["center-lat","center-long","polygon"]][0:33]


    polygons = []
    for polygon in df_labels["polygon"]:
        polygon_temp = []
        for coordinates in json.loads(polygon)["coordinates"]:
            for coordinate in coordinates:
                polygon_temp.append(tuple(coordinate))
            polygons.append(Polygon(polygon_temp))

    gdf_series = gpd.GeoSeries(polygons)
    gdf = gpd.GeoDataFrame(gdf_series,geometry=0)
    gdf["geometry"] = gdf[0]
    gdf = gdf.drop(columns=[0])
    return gdf

In [15]:
csv_loc = "/Users/purgatorid/Documents/GitHub/canopy-gis/data_collection/data/labelled/labels_Misha_v2.csv"

gdf = csv_to_gdf(csv_loc)

In [7]:
def convert_rasters(src_dir, dest_dir, epsg_format='EPSG:4326', windows=False):
    """Converts the rasters in the src_dir into a different EPSG format,
    keeping the same folder structure and saving them in the dest_dir."""

    src_dir = add_trailing_slash(src_dir)
    dest_dir = add_trailing_slash(dest_dir)
    
    # If the output folder doesn't exist, create it
    create_dir(dest_dir)

    input_files = glob(src_dir + '*/*.jp2')
    # Keep track of how many files were converted
    n = 1
    total = len(input_files)
    
    for f in input_files:
        print(f'processing file {n} of {total}')
        n += 1
        
        # The way we've set it up, we save each product into a numbered folder,
        # depending on which layer it's in. To keep this structure, we need to
        # pull out the folder number from the file path.
        # How exactly to do this depends on if you're using Windows or not,
        # since the path conventions are different.
        if windows:
            folder_num = f.split('\\')[-2]
            filename = f.split('\\')[-1]
        else:
            folder_num = f.split('/')[-2]
            filename = f.split('/')[-1]
        output_folder = dest_dir + folder_num + '/'
        
        
        # If the respective grouping folders are not available 
        create_dir(output_folder)
        
        output_filepath = output_folder + filename
        
        print(output_filepath)
        print(f)

        # Finally, we convert
        converted = gdal.Warp(output_filepath, [f],format='GTiff',
                              dstSRS=epsg_format, resampleAlg='near')
        converted = None
        
    print('Finished')
    

In [10]:
src_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_ordered"
dest_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_ordered_warped"

convert_rasters(src_dir, dest_dir)

processing file 1 of 4
/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_ordered_warped/1/processed_T33NYC_20200927T090731_TCI_10m.jp2
/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_ordered/1/processed_T33NYC_20200927T090731_TCI_10m.jp2
processing file 2 of 4
/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_ordered_warped/1/processed_T34MBB_20200926T084719_TCI_10m.jp2
/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_ordered/1/processed_T34MBB_20200926T084719_TCI_10m.jp2
processing file 3 of 4
/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_ordered_warped/1/processed_T34NEF_20200906T084559_TCI_10m.jp2
/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_ordered/1/processed_T34NEF_20200906T084559_TCI_10m.jp

In [28]:
def make_full_virtual_raster(src_dir, dest_dir, num_layers=2):
    """Combines the rasters in the src_dir into a single virtual raster
    with proper prioritization. This is saved into the dest_dir.
    Make sure the num_layers variable is the same as the number of tile layers
    in your src_dir."""
    
    src_dir = add_trailing_slash(src_dir)
    dest_dir = add_trailing_slash(dest_dir)
    
    # If the output folder doesn't exist, create it
    create_dir(dest_dir)
    
    
    
    for layer in range(1, num_layers+1):
        print('Making Layer', layer)
        
        # Get the filenames from the layer in question
        filenames = glob(src_dir + f'{layer}/*.jp2', recursive=True)
        
        output_file = dest_dir + f'Layer{layer}.vrt'
    
        vrt = gdal.BuildVRT(output_file, filenames, resolution='average', resampleAlg='nearest', srcNodata=0)
    
        vrt.FlushCache()
    
    print('Making full raster')

    # To make the full raster, we combine every layer. Do it in reverse order because (I believe)
    # the last items in the list are prioritized.

    input_files = [dest_dir + f'Layer{i}.vrt' for i in reversed(range(1, num_layers+1))]
    
    output_file = dest_dir + 'full.vrt'

    vrt = gdal.BuildVRT(output_file, input_files, resolution='average', resampleAlg='nearest', srcNodata=0)

    vrt.FlushCache()

    print('Finished')

In [29]:
src_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_ordered_warped"
dest_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_master_raster"

make_full_virtual_raster(src_dir, dest_dir)


Making Layer 1
Making Layer 2
Making full raster
Finished


In [30]:
def vrt_to_tif(output_file,src_file):

    translate = gdal.Translate("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_master_raster/full_tif.tif", "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/test_master_raster/full.vrt",
                               format='GTiff')
    translate.FlushCache()

In [16]:
def export_aoi_polygon_rasters(gdf,master_raster_path,output_dir):
    
    output_dir = add_trailing_slash(output_dir) 

    src_raster_file = rio.open(master_raster_path)
    
    for index in range(gdf.shape[0]):
        crop_extent = gdf.loc[[index],"geometry"]


        raster_crop, raster_meta = es.crop_image(src_raster_file, crop_extent)

        # Update the metadata to have the new shape (x and y and affine information)
        raster_meta.update({"driver": "GTiff",
                         "height": raster_crop.shape[1],
                         "width": raster_crop.shape[2],
                         "transform": raster_meta["transform"]})

        # generate an extent for the newly cropped object for plotting
        cr_ext = rio.transform.array_bounds(raster_meta['height'], 
                                                    raster_meta['width'], 
                                                    raster_meta['transform'])
        
        

        bound_order = [0,2,1,3]
        cr_extent = [cr_ext[b] for b in bound_order]

        # mask the nodata
        raster_crop_ma = np.ma.masked_equal(raster_crop, 0) 


        # output_path
        outpath = out_base_path + str(index+1) + '.tif'

        # Check if directory exists
        if not os.path.isdir(out_base_path):
            os.mkdir(out_base_path)


        # Export cloud-masked TCI file
        with rio.open(outpath, 'w', **raster_meta) as outf:
            outf.write(raster_crop_ma)

In [17]:
master_raster_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Master_Rasters/msk_geotiff_full.tif"
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops_Test/"

export_aoi_polygon_rasters(gdf,master_raster_path,output_dir)

KeyboardInterrupt: 

In [69]:
def tif_to_jpg(in_dir,out_dir):
    
    
    
    in_dir = add_trailing_slash(in_dir)
    
    out_dir = add_trailing_slash(out_dir)
    
    # If the output folder doesn't exist, create it
    
    create_dir(out_dir)
    

    # Export Polygons from TIF to  JPEG

    tif_list = glob(in_dir + "*.tif",recursive=True)
    
    for tif_path in tif_list:
        base_filename = tif_path.split("/")[-1].split(".")[0]
        im = Image.open(tif_path)
        im.thumbnail(im.size)
        im.save(out_dir + base_filename + ".jpg", "JPEG", quality=100)
    

In [75]:
in_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Individual_Polygons/TIF/"
out_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Individual_Polygons/JPG_2"

tif_to_jpg(in_dir,out_dir)

# Sandbox

In [31]:
within_basin_tile_list = ['33MVV',
 '33NTE',
 '33NWE',
 '32NRK',
 '33NUB',
 '32NPG',
 '33NUC',
 '33NUG',
 '32NMK',
 '33NZD',
 '33NVE',
 '33NXD',
 '33NVD',
 '33NTG',
 '34NDF',
 '34MCE',
 '35NLB',
 '33NXE',
 '34MBC',
 '34MAC',
 '33NTC',
 '33NVF',
 '34MCB',
 '34NCH',
 '32NMM',
 '34NEH',
 '33MWV',
 '34NDG',
 '33NTB',
 '34NEF',
 '33NUD',
 '34NAJ',
 '33NWC',
 '34MAB',
 '34MBB',
 '32NQH',
 '33NYC',
 '33NWA',
 '33NUF',
 '34NDH',
 '33NWD',
 '33NUE',
 '33NWB',
 '33NVC',
 '33NYB',
 '33NTD',
 '34NFH',
 '32NNM',
 '32NRH',
 '33NXB',
 '33NYD',
 '33NVA',
 '33NSE',
 '33NSD',
 '34NFG',
 '34NGG',
 '33MZT',
 '32NQG',
 '33NSF',
 '33NXC',
 '33NYE',
 '33NTF',
 '32NML']

In [32]:
len(wihthin_basin_tile_list)

NameError: name 'wihthin_basin_tile_list' is not defined

In [50]:
# bounding_box = [[[5.493164,8.276727],[5.449219,-5.703448],[31.376953,-4.959615],[31.157227,8.711359],[5.493164,8.276727]]]

# Congo Basin Bounding Box (updated)
search_bbox = BBox(bbox=[7.3224651059994361,-7.1358345205706115, 31.0068991504288292, 6.9323631449175238], crs=CRS.WGS84)

#Additional Params
bands = ['R10m/TCI']

In [51]:
def shub_lookup_tiles(bounding_box=search_bbox,tile_list=within_basin_tile_list,search_time_interval = ('2019-01-01T00:00:00', '2020-12-31T23:59:59'),
                   product_type = DataSource.SENTINEL2_L2A):
    
    #Misha's Tiles of Interest
    search_bbox = bounding_box

    search_time_interval = ('2019-01-01T00:00:00', '2020-12-31T23:59:59')
    wfs_iterator = WebFeatureService(
        search_bbox,
        search_time_interval,
        data_source=product_type,
        config=config
    )
    results = wfs_iterator.get_tiles()
    df = pd.DataFrame(results, columns=['Tilename','Date','AmazonID'])
#     df_tiles_of_interest = df[df["Tilename"].isin(tile_list)]
#     df2 = df_tiles_of_interest.groupby('Tilename').head(1)
#     output2 = list(df2.itertuples(index=False,name=None))
    return df

In [52]:
df = shub_lookup_tiles()

In [41]:
pd.set_option('display.max_rows', 100)

# df.to_csv("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/tile_lists/shub_tile_list/tile_ids_full_v2.csv",index=False)

In [36]:
bounding_box_tiles = list(df["Tilename"].unique())

In [37]:
tile_list_txt = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/tile_lists/shub_tile_list/tile_ids_2.txt"

In [53]:
df2 = df.groupby('Tilename').head(1)

In [56]:
df2["Tilename"].to_csv("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/tile_lists/shub_tile_list/tile_ids_full_v2.csv",index=False)

In [44]:
output2 = list(df2.itertuples(index=False,name=None))

In [46]:
def shub_download_tiles(results_list,output_dir,bands=["R10m/TCI"],product_type = DataSource.SENTINEL2_L2A):
    
    #Additional Params
    bands = bands
    
    output_dir = add_trailing_slash(output_dir)
    
    
    for tile in results_list:
        tile_name, time, aws_index = tile

        #Download SAFE Files
        request = AwsTileRequest(
            tile=tile_name,
            time=time,
            bands=bands, 
            aws_index=aws_index,
            data_folder=output_dir,
            data_source=product_type,
            safe_format=True
        )

        request.save_data(redownload=True)

In [49]:
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Full_Congo_Tile_Pull"

shub_download_tiles(output2,output_dir,bands=["R10m/TCI"],product_type = DataSource.SENTINEL2_L2A)

/Users/purgatorid/opt/anaconda3/envs/pc_staging/lib/python3.8/site-packages/sentinelhub/aws.py:164: SHUserWarning: Products with baseline 02.14 are not officially supported in sentinelhub-py. If you notice any errors in naming structure of downloaded data please report an issue at https://github.com/sentinel-hub/sentinelhub-py/issues. Pull requests are also very appreciated
  warnings.warn(message, category=SHUserWarning)
/Users/purgatorid/opt/anaconda3/envs/pc_staging/lib/python3.8/site-packages/sentinelhub/aws.py:164: SHUserWarning: Products with baseline 02.14 are not officially supported in sentinelhub-py. If you notice any errors in naming structure of downloaded data please report an issue at https://github.com/sentinel-hub/sentinelhub-py/issues. Pull requests are also very appreciated
  warnings.warn(message, category=SHUserWarning)
/Users/purgatorid/opt/anaconda3/envs/pc_staging/lib/python3.8/site-packages/sentinelhub/aws.py:164: SHUserWarning: Products with baseline 02.14 are 